# Pre-analysis of the BlueGene/L dataset 
This notebook uses data mining techniques to help better understand the data

### About the dataset

BGL is an open dataset of logs collected from a BlueGene/L supercomputer system at Lawrence Livermore National Labs (LLNL) in Livermore, California, with 131,072 processors and 32,768GB memory. The log contains alert and non-alert messages identified by alert category tags. In the first column of the log, "-" indicates non-alert messages while others are alert messages. The label information is amenable to alert detection and prediction research. It has been used in several studies on log parsing, anomaly detection, and failure prediction.

### Structure of log

Logs are structure following the format "LABEL TIMESTAMP DATE NODE DATE-FULL NODE(again) TYPE COMPONENT LEVEL CONTENT". For example:

"- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected"

In [1]:
# dependencies
from utils.parser import parse_logs
from utils.paths import project_root

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [14]:
# loads parsed logs if needed
parse_logs(f'{project_root()}/data', 'BGL.log', f'{project_root()}/data/parsed')
#TODO: for now lines with no content are skipped. Later define if stays this way

Skipping: Output file 'cleaned_BGL_structured.csv' already exists.


In [3]:
df = pd.read_csv(f'{project_root()}/data/parsed/cleaned_BGL_structured.csv')
df.head()

,LineId,Label,Timestamp,Date,Node,Time,NodeRepeat,Type,Component,Level,Content,EventId,EventTemplate,ParameterList
0,1,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
1,2,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
2,3,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
3,4,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
4,5,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]


## Understanding Features
Labels tell what the type of the event that geterated the log. Most of the logs have a "-" in the label, meaning there was not problem identified in that log. According to the article "What Supercomputers Say: A Study of Five System Logs
" (https://ieeexplore.ieee.org/document/4273008), the types can be split in the following categories:

![labels](../../images/labels.png) 


In [4]:
# anomalies distribution
df['Label'].value_counts()

Label
-            4365033
KERNDTLB      152734
KERNSTOR       63491
APPSEV         49651
KERNMNTF       31531
KERNTERM       23338
KERNREC         6145
APPREAD         5983
KERNRTSP        3983
APPRES          2370
APPUNAV         2048
APPTO           1991
KERNMICRO       1503
APPOUT           816
KERNMNT          720
APPBUSY          512
KERNMC           342
APPCHILD         320
KERNSOCK         209
KERNPOW          192
LINKIAP          166
APPALLOC         144
KERNSERV          94
MASABNORM         37
LINKDISC          24
KERNPAN           18
KERNCON           16
LINKPAP           14
KERNNOETH         14
MONPOW            12
MASNORM           10
APPTORUS          10
KERNPROG           5
MMCS               3
KERNRTSA           3
KERNFLOAT          3
MONNULL            2
LINKBLL            2
KERNBIT            1
KERNEXT            1
MONILL             1
KERNTLBE           1
Name: count, dtype: int64

Similar to labels, we have level which can tell us the gravity of the report of the log. It can go from just an INFO log to a failure or kill

In [5]:
df['Level'].value_counts()

Level
INFO            3701880
FATAL            854658
ERROR            112355
WARNING           23357
SEVERE            19213
FAILURE            1714
Kill                306
single                4
microseconds          4
0x00544eb8            2
Name: count, dtype: int64

In [6]:
df['Content'].value_counts().head(20)

Content
data TLB error interrupt                                                                                                                                                                                                                                                                                                                                                                             152734
0 microseconds spent in the rbs signal handler during 0 calls. 0 microseconds was the maximum time for a single instance of a correctable ddr.                                                                                                                                                                                                                                                       135005
instruction cache parity error corrected                                                                                                                                                                

### Parsing

The logparser library was used to parse the log events, removing variable data from the content like ip addresses or variable names and keeping the core of the message. This can be used to create the bag-of-logs embedding

In [11]:
df['EventId'].nunique()

390

In [12]:
df['EventTemplate'].nunique()

390

In [13]:
df['EventTemplate'].value_counts().head(20)

EventTemplate
generating <*>                                                                                                                                                             1706751
iar <*> dear <*>                                                                                                                                                            442966
<*> double-hummer alignment exceptions                                                                                                                                      295764
<*> floating point alignment exceptions                                                                                                                                     267725
CE sym <*> at <*> mask <*>                                                                                                                                                  201206
data TLB error interrupt                                                                   